In [1]:
import requests
import json
import pandas as pd

# IMF API
The purpose of this notebook is to investigate the [IMF's JSON RESTful Web Service](http://datahelp.imf.org/knowledgebase/articles/667681-using-json-restful-web-service)
According to the IMF's documentation the following methods are available:
* Dataflow 
* DataStructure 
* CompactData
* MetadataStructure
* GenericMetadata 
* CodeList
* MaxSeriesInResult

Since I did not find their documentation very helpful, I decided to write a notebook to get a clearer picture of the data available through those methods. I will also use it to create a IMF client to be used in the IMF dashboard project. I will only be looking into the first three methods since they seem the only ones needed to access data according to the IMF's documentation

I will go method by method, calling it and looking at the answer, transforming and cleaning data to dataframe in the process.

## Dataflow:

In [2]:
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/Dataflow'
r = requests.get(url)
data = r.json()
data

{'Structure': {'@xmlns:xsd': 'http://www.w3.org/2001/XMLSchema',
  '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  '@xmlns': 'http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message',
  '@xsi:schemaLocation': 'http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message https://registry.sdmx.org/schemas/v2_0/SDMXMessage.xsd',
  'Header': {'ID': 'dc8c3507-bebf-4fda-9d55-f3d4112f13c4',
   'Test': 'false',
   'Prepared': '2019-02-26T11:01:44',
   'Sender': {'@id': '1C0',
    'Name': {'@xml:lang': 'en', '#text': 'IMF'},
    'Contact': {'URI': 'http://www.imf.org',
     'Telephone': '+ 1 (202) 623-6220'}},
   'Receiver': {'@id': 'ZZZ'}},
  'Dataflows': {'Dataflow': [{'@id': 'DS-FAS',
     '@version': '1.0',
     '@agencyID': 'IMF',
     '@isFinal': 'true',
     '@xmlns': 'http://www.SDMX.org/resources/SDMXML/schemas/v2_0/structure',
     'Name': {'@xml:lang': 'en', '#text': 'Financial Access Survey (FAS)'},
     'KeyFamilyRef': {'KeyFamilyID': 'FAS', 'KeyFamilyAgencyID': 'IMF'}}

In [3]:
data.keys()  # It starts off with one key

dict_keys(['Structure'])

In [4]:
data['Structure'].keys()  # In this key we found 6 keys were Dataflow contains 
                          # a list of dictionaries

dict_keys(['@xmlns:xsd', '@xmlns:xsi', '@xmlns', '@xsi:schemaLocation', 'Header', 'Dataflows'])

In [5]:
dataflows = pd.DataFrame.from_dict(data['Structure']['Dataflows']['Dataflow'])
# I will perform some transformations to retain the information that I though useful.
dataflows['Description'] = dataflows.Name.apply(lambda d: d['#text'])  # Access subdict. 
dataflows['KeyFamilyID'] = dataflows.KeyFamilyRef.apply(lambda d: d['KeyFamilyID'])  # Access subdict. 
dataflows['KeyFamilyAgencyID'] = dataflows.KeyFamilyRef.apply(lambda d: d['KeyFamilyAgencyID'])  # Access subdict. 
dataflows = dataflows[['Description', 'KeyFamilyID', 'KeyFamilyAgencyID']]
dataflows.sample(5)

Description       KeyFamilyID  \
125  Government Finance Statistics Yearbook (GFSY 2...        GFSIBS2017   
157  International Financial Statistics (IFS), 2018...       IFS_2018M11   
30   Sub-Saharan Africa Regional Economic Outlook (...      AFRREO201504   
64                                    Gender Budgeting  GENDER_BUDGETING   
66                         Financial Development Index               FDI   

    KeyFamilyAgencyID  
125               IMF  
157               IMF  
30                IMF  
64                IMF  
66                IMF

In [6]:
dataflows.describe()

Description       KeyFamilyID  \
count                                                 173               173   
unique                                                173               173   
top     Sub-Saharan Africa Regional Economic Outlook (...  GENDER_BUDGETING   
freq                                                    1                 1   

       KeyFamilyAgencyID  
count                173  
unique                 1  
top                  IMF  
freq                 173

The dataflows method returns the available data sources. For example: the _Financial Access Survey_,  the _Fiscal Monitor_ ,the _Direction of Trade Statistics_ or the _International Financial Statistics_.
There are at the time of writting this notebbok 173 data sources all belonging to one "Family Agency" the IMF.

## Data Structure
As the documentation tells us, the DataStructure returns "the structure of the dataset". So lets see what that looks like. As an example lets look at the International Financial Statistics (IFS)

In [7]:
dataset = 'IFS'
url = "http://dataservices.imf.org/REST/SDMX_JSON.svc/DataStructure/{}".format(dataset)
r = requests.get(url)
ds_struct = r.json()
ds_struct.keys()

dict_keys(['Structure'])

In [8]:
ds_struct['Structure'].keys()   # Seems to follow a similar format as the dataflows.

dict_keys(['@xmlns:xsd', '@xmlns:xsi', '@xmlns', '@xsi:schemaLocation', 'Header', 'CodeLists', 'Concepts', 'KeyFamilies'])

In [9]:
# The information seems inside this dictionary as a list of dictionaries  
ds_struct = pd.DataFrame().from_dict(ds_struct['Structure']['CodeLists']['CodeList']) 

In [10]:
ds_struct = ds_struct.set_index('@id')

In this level there is a list of the kind of codes available. It contains the metadata for how the data is stored. For example: CL_UNIT_MULT contains a dictionary with the relation between a value for example 2 and a description "Hundreds". The same applies to the rest.
The most usefull seem to be CL_FREQ, CL_AREA_IFS and CL_INDICATOR_IFS since they will be needed (as we will see) to obtain the data for an specific indicator.

In [11]:
pd.DataFrame().from_dict(ds_struct.loc[1]['Code']).head()

TypeError: cannot do label indexing on <class 'pandas.core.indexes.base.Index'> with these indexers [1] of <class 'int'>

In [ ]:
pd.DataFrame().from_dict(ds_struct.loc[2]['Code']).head()

In [ ]:
pd.DataFrame().from_dict(ds_struct.loc[3]['Code']).head()

In [ ]:
codes = pd.DataFrame.from_dict(ds_struct.loc[3]['Code'])
codes['Description'] = codes.Description.apply(lambda dct: dct['#text'])
codes.rename(columns={'@value': 'code'}, inplace=True)

In [ ]:
codes.head()

## CompactData
According to the documentation, the compactData method : "CompactData method returns the compact data message". Again not very helpful. The format for calling it is:
`http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/{database ID}/{frequency}.{item1 from
dimension1}+{item2 from dimension1}+{item N from dimension1}.{item1 from
dimension2}+{item2 from dimension2}+{item M from dimension2}?startPeriod={start
date}&endPeriod={end date}`

In [ ]:
dataflow = 'IFS'
frequency = 'M'
start_period = '1950'
end_period = '2017'
country = 'US'
indicator = 'FILR_PA'
imf = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
url = '{}CompactData/{}/{}.{}.{}.?startPeriod={}&endPeriod={}'.format(imf, dataflow, frequency, country, indicator,
                                                                      start_period, end_period)
r = requests.get(url)
c_data = r.json()

In [ ]:
pd.DataFrame.from_dict(c_data['CompactData']['DataSet']['Series']['Obs'])['@OBS_VALUE'].apply(lambda s: float(s)).plot()

If available, the data will be stores in CompactData>DataSet>Series>Obs however many combinations of countries, indicators and frequencies are not available.